# Aula 2 — Introdução ao `pandas` para Epidemiologia

> **Curso:** Python para Epidemiologistas — Aula 2  
> **Foco:** criação/inspeção de DataFrames, seleção e filtros (`query`/máscaras), criação de colunas (`assign`) e resumo com `groupby().agg()`; mini‑EDA com dados sintéticos + **exemplo** de leitura de dados do **OpenDataSUS (SIVEP‑Gripe)**.

## Objetivos de aprendizagem
* Criar e inspecionar `DataFrame`s (`shape`, `info`, `describe`, `value_counts`).
* Selecionar colunas, filtrar linhas com **máscaras booleanas** e com `DataFrame.query`.
* Criar e transformar variáveis com `assign` e resumir com `groupby().agg`.
* Realizar uma mini‑exploração de um *line list* epidemiológico (frequências por **região** e **idade**).
* Carregar dados **remotos** do **OpenDataSUS (SIVEP‑Gripe)** *com fallback* para dados sintéticos.

## Agenda (2 horas)
| Horário | Conteúdo |
|---|---|
| **0:00–0:10** | Abertura, contexto, como usar o Colab |
| **0:10–0:25** | Criação e **inspeção** de DataFrames |
| **0:25–0:40** | **Seleção** de colunas e **filtragem** (máscaras / `query`) |
| **0:40–0:55** | **Criação de colunas** com `assign` e **resumos** com `groupby().agg()` |
| **0:55–1:05** | Pausa |
| **1:05–1:25** | **Mini‑EDA** do *line list* (frequências por região/idade) e **exemplo DATASUS** |
| **1:25–1:45** | Exercícios finais (3) com dificuldade progressiva |
| **1:45–2:00** | Soluções, checklist e próximos passos |

## Pré‑requisitos
Ter concluído a **Aula 1** (Fundamentos de Python), entendendo tipos de dados, estruturas básicas, loops, funções e leitura de CSV.

## Como usar este notebook no Colab
1. Clique no botão de Colab acima ou faça upload do arquivo diretamente no Colab.
2. Execute as células em ordem (Ctrl/Cmd + Enter ou botão de play).
3. Para salvar sua cópia: **File → Save a copy in Drive**.
4. Para reiniciar o kernel: **Runtime → Restart runtime**.


In [ ]:
#@title Instalar/Carregar dependências
# Usamos bibliotecas já disponíveis no Colab (pandas, numpy, matplotlib).
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

# Semente para resultados reprodutíveis
rng = np.random.default_rng(42)
print("Ambiente carregado")

## 1) Dados: sintético + OpenDataSUS

Trabalharemos com um **line list** sintético contendo variáveis epidemiológicas básicas e um **exemplo** de carregamento de dados do **OpenDataSUS** (SIVEP‑Gripe) para demonstrar a leitura remota. Caso a leitura remota falhe, usaremos apenas os dados sintéticos.

### Dicionário de variáveis (sintético)
* `id`: identificador do caso.
* `age`: idade em anos.
* `sex`: sexo (`'F'` ou `'M'`).
* `region`: macrorregião brasileira (`Norte`, `Nordeste`, `Sudeste`, `Sul`, `Centro-Oeste`).
* `onset_date`: data de início dos sintomas.
* `report_date`: data de notificação.
* `hospitalized`: status de hospitalização (`internado`/`nao_internado`).

> No dataset do **SIVEP‑Gripe**, os nomes das colunas são diferentes (ex.: `CS_SEXO`, `NU_IDADE_N`, etc.). A função de leitura abaixo tenta normalizá‑las para termos consistentes.


In [10]:
# Funções utilitárias
def generate_synthetic_line_list(n=1000, seed=42):
    rng = np.random.default_rng(seed)
    ids = np.arange(1, n+1)
    ages = rng.integers(0, 90, size=n)
    sex = rng.choice(['F', 'M'], size=n, p=[0.53, 0.47])
    regions = rng.choice(['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste'], size=n, p=[0.08,0.28,0.43,0.14,0.07])
    start = np.datetime64('2023-01-01')
    onset = start + rng.integers(0, 300, size=n).astype('timedelta64[D]')
    report_delay = rng.integers(0, 10, size=n).astype('timedelta64[D]')
    report = onset + report_delay
    hosp = rng.choice(['internado', 'nao_internado'], size=n, p=[0.18, 0.82])
    return pd.DataFrame({
        'id': ids,
        'age': ages,
        'sex': sex,
        'region': regions,
        'onset_date': pd.to_datetime(onset),
        'report_date': pd.to_datetime(report),
        'hospitalized': hosp
    })

# Tentativa de carregar SIVEP-Gripe
import pandas as pd
import numpy as np

def try_load_opendatasus_srag(sample_n=20000):
    url = 'https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/2023/INFLUD23-26-06-2025.csv'
    try:
        df_raw = pd.read_csv(url, sep=';', encoding='latin1', nrows=sample_n, low_memory=False)
        colmap = {}
        for c in ['CS_SEXO','SEXO']:
            if c in df_raw.columns:
                colmap[c] = 'sex'
        for c in ['NU_IDADE_N','NU_IDADE']:
            if c in df_raw.columns:
                colmap[c] = 'age'
        for c in ['DT_SIN_PRI','DT_SINTOMA']:
            if c in df_raw.columns:
                colmap[c] = 'onset_date'
        for c in ['DT_NOTIFIC','DT_NOTIF']:
            if c in df_raw.columns:
                colmap[c] = 'report_date'
        for c in ['SG_UF_NOT','SG_UF']:
            if c in df_raw.columns:
                colmap[c] = 'uf'
        df = df_raw.rename(columns=colmap).copy()
        if 'uf' in df.columns:
            uf_to_region = {
                'AC':'Norte','AP':'Norte','AM':'Norte','PA':'Norte','RO':'Norte','RR':'Norte','TO':'Norte',
                'AL':'Nordeste','BA':'Nordeste','CE':'Nordeste','MA':'Nordeste','PB':'Nordeste','PE':'Nordeste','PI':'Nordeste','RN':'Nordeste','SE':'Nordeste',
                'ES':'Sudeste','MG':'Sudeste','RJ':'Sudeste','SP':'Sudeste',
                'PR':'Sul','RS':'Sul','SC':'Sul',
                'DF':'Centro-Oeste','GO':'Centro-Oeste','MT':'Centro-Oeste','MS':'Centro-Oeste'
            }
            df['region'] = df['uf'].map(uf_to_region)

        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
        if 'sex' in df.columns:
            df['sex'] = df['sex'].astype(str).str.upper().str[0].replace({'F':'F','M':'M'})
        for dcol in ['onset_date','report_date']:
            if dcol in df.columns:
                df[dcol] = pd.to_datetime(df[dcol], errors='coerce', dayfirst=True)
        hosp_cols = [c for c in df_raw.columns if 'HOSP' in c.upper() or 'UTI' in c.upper()]
        if hosp_cols:
            df['hospitalized'] = np.where(df_raw[hosp_cols].notna().any(axis=1), 'internado', 'nao_internado')
        else:
            df['hospitalized'] = 'nao_internado'
        keep = [c for c in ['age','sex','region','onset_date','report_date','hospitalized'] if c in df.columns]
        if not keep:
            return None
        return df[keep].dropna(how='all')
    except Exception as e:
        print('Falha ao carregar OpenDataSUS:', e)
        return None

# Carrega com fallback
_df_remote = try_load_opendatasus_srag(sample_n=20000)
if _df_remote is None or _df_remote.empty:
    print('Usando dados SINTÉTICOS (fallback).')
    df = generate_synthetic_line_list(n=1200, seed=42)
else:
    print('Dados do OpenDataSUS (amostra) carregados.')
    df = _df_remote.copy()

# Visualiza cabeçalho
df.head()


           NU_NOTIFIC report_date  SEM_NOT  onset_date  SEM_PRI  uf                       ID_REGIONA  CO_REGIONA  \
0        316004370301  2023-02-07        6  2023-02-05        6  SP              GVE VII SANTO ANDRE      1332.0   
1        316153805701  2023-02-28        9  2023-01-17        3  SP              GVE VII SANTO ANDRE      1332.0   
2        316321420027  2023-05-02       18  2023-04-14       15  PR               02RS METROPOLITANA      1356.0   
3        316384665515  2023-03-09       10  2023-03-05       10  SP              GVE VII SANTO ANDRE      1332.0   
4      31643223416015  2023-01-21        3  2023-01-17        3  MG                     JUIZ DE FORA      1452.0   
...               ...         ...      ...         ...      ...  ..                              ...         ...   
19995  31676311526279  2023-02-13        7  2023-01-11        2  MG                           PASSOS      1455.0   
19996  31676311563354  2023-02-12        7  2023-02-10        6  MA  REG

,id,age,sex,region,onset_date,report_date,hospitalized
0,1,8,F,Sudeste,2023-07-05,2023-07-11,nao_internado
1,2,69,M,Sudeste,2023-09-06,2023-09-08,nao_internado
2,3,58,M,Nordeste,2023-07-12,2023-07-14,internado
3,4,39,M,Sudeste,2023-06-28,2023-06-29,nao_internado
4,5,38,M,Sudeste,2023-05-21,2023-05-21,nao_internado


## 2) Criação e **inspeção** de DataFrames

Recomendações iniciais para explorar os dados:

* `df.shape`, `df.dtypes`, `df.info()` — informações de estrutura.
* `df.head()`, `df.tail()` — primeiras e últimas linhas.
* `df.describe()` — estatísticas descritivas.
* `df['col'].value_counts()` — frequências de valores categóricos.


In [ ]:
# Inspeção rápida
print('shape:', df.shape)
print(df.dtypes)
df.info()
print(df.describe(include='all', datetime_is_numeric=True).T.head(10))

if 'sex' in df.columns:
    print(df['sex'].value_counts(dropna=False))


**Exercício (após Seção 2) — Aquecimento**  
Calcule:
1) o número de valores ausentes em cada coluna (`isna().sum()`);
2) a proporção de valores ausentes por coluna.

*Escreva seu código na célula abaixo.*


In [ ]:
# 1) contagem e proporção de NAs por coluna
na_count = df.isna().sum()
na_prop = df.isna().mean()
print('NAs por coluna:')
print(na_count)
print('Proporção de NAs por coluna:')
print(na_prop)


<details><summary><strong>Solução</strong></summary>

```python
na_count = df.isna().sum()
na_prop  = df.isna().mean()
print(na_count)
print(na_prop)
```

</details>


## 3) **Seleção** de colunas e **filtragem** de linhas

Formas comuns de seleção:

* Selecionar colunas por lista: `df[['col1','col2']]`.
* Filtrar linhas com máscaras booleanas: `df[df['age'] >= 60]`.
* Filtrar com `query`: `df.query('age >= 60 and sex == "F"')` — legível e comparável ao `filter()` do `dplyr`.


In [ ]:
# Seleção de colunas
subset_cols = [c for c in ['age','sex','region','onset_date','hospitalized'] if c in df.columns]
df_sel = df[subset_cols].copy()
df_sel.head()


In [ ]:
# Filtragem com máscara booleana: idade >= 60
if 'age' in df.columns:
    idosos = df_sel[df_sel['age'] >= 60]
    print('Registros com idade >= 60:', idosos.shape[0])
    print(idosos.head())


In [ ]:
# Filtragem com query: mulheres 20–39
if all(c in df_sel.columns for c in ['age','sex']):
    mulheres_20a39 = df_sel.query('sex == "F" and 20 <= age <= 39')
    print('Mulheres 20–39:', mulheres_20a39.shape[0])
    print(mulheres_20a39.head())


**Exercício (após Seção 3)**  
Usando `df_sel`, selecione apenas `age`, `sex` e `region` e filtre **homens (`"M"`) com idade < 30**. Mostre as 5 primeiras linhas.


In [ ]:
# Filtrar homens com idade < 30
if all(c in df_sel.columns for c in ['age','sex','region']):
    res = df_sel[['age','sex','region']].query('sex == "M" and age < 30')
    print(res.head())
else:
    print('Colunas necessárias não disponíveis.')


<details><summary><strong>Solução</strong></summary>

```python
res = df_sel[['age','sex','region']]
out = res.query('sex == "M" and age < 30')
out.head()
```

</details>


## 4) **Criação de colunas** com `assign` e **resumos** com `groupby().agg()`

* `assign(nova_col = expressão)`: cria ou transforma colunas de forma encadeável.
* `groupby(chaves).agg(funcs)`: resume dados por grupos (contagens, médias, proporções…).


In [ ]:
# Criar faixa etária e resumir por região
def cut_age(x):
    if pd.isna(x):
        return np.nan
    bins = [-1, 4, 11, 17, 29, 39, 49, 59, 69, 79, 200]
    labels = ['0-4','5-11','12-17','18-29','30-39','40-49','50-59','60-69','70-79','80+']
    return pd.cut([x], bins=bins, labels=labels)[0]

if 'age' in df.columns:
    df_aug = df.assign(age_group = df['age'].apply(cut_age))
else:
    df_aug = df.copy()

if all(c in df_aug.columns for c in ['region','age_group']):
    resumo = (df_aug
              .groupby(['region','age_group'], dropna=False)
              .agg(casos=('age','size'))
              .reset_index()
              .sort_values(['region','age_group']))
    print(resumo.head(12))


**Exercício (após Seção 4)**  
Crie uma coluna booleana `is_elderly` indicando **idade ≥ 60** e calcule, **por região**, a **proporção** de `is_elderly` (use `mean()` sobre booleanos).


In [ ]:
# Proporção de idosos por região
if 'age' in df.columns and 'region' in df.columns:
    tmp = (df.assign(is_elderly = df['age'] >= 60)
             .groupby('region')
             .agg(prop_elderly=('is_elderly','mean'), n=('is_elderly','size'))
             .reset_index()
             .sort_values('prop_elderly', ascending=False))
    print(tmp)
else:
    print('Colunas necessárias ausentes.')


<details><summary><strong>Solução</strong></summary>

```python
out = (df.assign(is_elderly = df['age'] >= 60)
         .groupby('region')
         .agg(prop_elderly=('is_elderly','mean'),
              n=('is_elderly','size'))
         .reset_index())
out.sort_values('prop_elderly', ascending=False)
```

</details>


## 5) Mini‑EDA: frequências por região e idade

Vamos calcular frequências por **região** e por **faixa etária** utilizando o DataFrame `df_aug`.


In [ ]:
# Frequências
if 'region' in df_aug.columns:
    freq_reg = df_aug['region'].value_counts(dropna=False).rename('casos').to_frame()
    print(freq_reg)
if 'age_group' in df_aug.columns:
    freq_age = df_aug['age_group'].value_counts(dropna=False).rename('casos').to_frame()
    print(freq_age)


## 6) Mapa mental **R ↔ Python**

| Tarefa (`dplyr` em R) | `pandas` (Python) |
|---|---|
| `select(col1, col2)` | `df[['col1','col2']]` |
| `filter(x > 0 & sexo == 'F')` | `df[(df['x'] > 0) & (df['sexo'] == 'F')]` ou `df.query('x > 0 and sexo == "F"')` |
| `mutate(nova = f(x))` | `df.assign(nova = f(df['x']))` |
| `summarise(n = n())` + `group_by(grp)` | `df.groupby('grp').agg(n=('col','size'))` |
| `arrange(desc(x))` | `df.sort_values('x', ascending=False)` |
| `rename(novo = antigo)` | `df.rename(columns={'antigo':'novo'})` |

> Operações de **joins** e **reshapes** serão abordadas na Aula 3.


## 7) Exercícios finais (3) — dificuldade progressiva

**E1. Filtrar registros por idade e sexo**  
Selecione `age`, `sex` e `region` e **filtre** pessoas **com `age ≥ 50` e `sex == "F"`**. Em seguida, conte o número de registros por região.

**E2. Criar categorias de faixa etária e contar casos**  
Utilizando a mesma categorização da seção 4, conte o número de casos por `age_group`.

**E3. Proporção de internados por região**  
Crie uma coluna booleana `is_hosp` (true se `hospitalized == 'internado'`) e calcule, **por região**, a **proporção** de internados e o número total de registros.


In [ ]:
# E1
subset = [c for c in ['age','sex','region'] if c in df.columns]
if len(subset) == 3:
    ans_e1 = (df[subset]
                .query('age >= 50 and sex == "F"')
                .groupby('region')
                .size()
                .rename('n')
                .reset_index()
                .sort_values('n', ascending=False))
    print(ans_e1)
else:
    print('Colunas necessárias para E1 ausentes.')


<details><summary><strong>Solução E1</strong></summary>

```python
ans_e1 = (df[['age','sex','region']]
            .query('age >= 50 and sex == "F"')
            .groupby('region')
            .size()
            .rename('n')
            .reset_index())
ans_e1.sort_values('n', ascending=False)
```

</details>


In [ ]:
# E2
def make_age_group(s):
    bins = [-1, 4, 11, 17, 29, 39, 49, 59, 69, 79, 200]
    labels = ['0-4','5-11','12-17','18-29','30-39','40-49','50-59','60-69','70-79','80+']
    return pd.cut(s, bins=bins, labels=labels)

if 'age' in df.columns:
    ans_e2 = (df.assign(age_group = make_age_group(df['age']))
                .groupby('age_group')
                .size()
                .rename('n')
                .reset_index()
                .sort_values('age_group'))
    print(ans_e2)
else:
    print('Coluna age ausente (E2).')


<details><summary><strong>Solução E2</strong></summary>

```python
bins = [-1, 4, 11, 17, 29, 39, 49, 59, 69, 79, 200]
labels = ['0-4','5-11','12-17','18-29','30-39','40-49','50-59','60-69','70-79','80+']
ans_e2 = (df.assign(age_group = pd.cut(df['age'], bins=bins, labels=labels))
            .groupby('age_group')
            .size()
            .rename('n')
            .reset_index())
ans_e2.sort_values('age_group')
```

</details>


In [ ]:
# E3
if all(c in df.columns for c in ['region','hospitalized']):
    ans_e3 = (df.assign(is_hosp = df['hospitalized'].eq('internado'))
                .groupby('region')
                .agg(prop_hosp=('is_hosp','mean'), n=('is_hosp','size'))
                .reset_index()
                .sort_values('prop_hosp', ascending=False))
    print(ans_e3)
else:
    print('Colunas necessárias para E3 ausentes.')


<details><summary><strong>Solução E3</strong></summary>

```python
ans_e3 = (df.assign(is_hosp = df['hospitalized'].eq('internado'))
            .groupby('region')
            .agg(prop_hosp=('is_hosp','mean'), n=('is_hosp','size'))
            .reset_index())
ans_e3.sort_values('prop_hosp', ascending=False)
```

</details>


## 8) Checklist de competências

* [ ] Sei inspecionar um DataFrame (`shape`, `info`, `describe`, `value_counts`).
* [ ] Sei selecionar colunas e filtrar linhas (máscaras e `query`).
* [ ] Sei criar colunas com `assign` e agregar com `groupby().agg`.
* [ ] Consigo reproduzir frequências por região e faixa etária.
* [ ] Consigo carregar uma amostra do OpenDataSUS ou usar o fallback sintético.

### O que praticar depois
* Experimentar filtros combinados (`and`, `or`, `in`) na função `query`.
* Criar funções auxiliares para categorizar idades de forma reaproveitável.
* Explorar outras colunas do dataset SIVEP‑Gripe (ex.: data de óbito, comorbidades).

### Referências
* Documentação do `pandas`: https://pandas.pydata.org/docs/
* `DataFrame.query`: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html
* Guia de `groupby`: https://pandas.pydata.org/docs/user_guide/groupby.html
* **OpenDataSUS – SRAG 2021 a 2025 (SIVEP‑Gripe)**: página oficial com links para os arquivos CSV e dicionário de dados.
